
# Notes:

## about intersection file from opendc
0. doesn't cover all intersections, but cost-effective
    - 355 STREETSEGID do not appear in the intersection file. E.g. 13150(-77.0613818238898 38.9236205926051), 6296(-77.07705117124307 38.925679831393)
    - some problems: not sure how this intersection is produced. E.g. segids=[4167, 8266], intersected at (38.905652137087 -77.01893411807097) ```intxn_dc[(intxn_dc.STREET1SEGID.isin(segids))|(intxn_dc.STREET2SEGID.isin(segids))][['STREET1SEGID','STREET2SEGID']]```from the map, there shouldn't be intersection of them, but there is (4167, 8266) edge. There should be a edge(4167,5326) but instead (5326, 4167); should be (4167, 3978), but no; but (8615, 4167) is there.
1. It's directed intersesction graph: (STREET1SEGID, STREET2SEGID) means STREET1SEGID can make a turn into STREET2SEGID. 
    - E.g. There is (3416L, 3523L) in the file, meaning you can turn into 3523 from 3416. But not otherwise.
2. The intersection file is more realistic than STREET SEGMENTS SHAPE FILE.
    - (STREET1SEGID, STREET2SEGID) could appear more than once. 
        - E.g. ```intxn_dc[intxn_dc.stpair_sorted==(9529, 12082)]```
    - Because it's realistic, the intersection points are not merely spatial join of the OPEN DC STREET SEGMENT File. 
        - E.g. ```intxn_dc[intxn_dc.stpair_sorted==(4766, 9755)]```
3. Almost all INTERSECTIONID has a unique latlon point, except:
    - ```intxn_dc[intxn_dc.INTERSECTIONID==12222].T```: same intersection with mirror different precision in coordinates
    - ```intxn_dc[intxn_dc.INTERSECTIONID==20330].T```: two different intersections
4. INTERSECTION_TYPE has 4 values: 'REGULAR': 17512, 'RAMP': 415, 'OVERPASS': 174, 'UNDERPASS': 175. OVERPASS and UNDERPASS are not accessible. 


In [4]:
import geopandas as gp
import pandas as pd
from src.constants import fn_segments_dc,fn_intxn_dc
from shapely.geometry import Point

In [5]:
data_dir = 'data/'
path_segs_dc = data_dir + fn_segments_dc
path_intxn_dc = data_dir + fn_intxn_dc


In [6]:
segs_dc = gp.read_file(path_segs_dc)
intxn_dc = pd.read_csv(path_intxn_dc)
intxn_dc['latlon'] = intxn_dc.apply(lambda x: (x.LATITUDE,x.LONGITUDE), axis=1)
intxn_dc['stpair_sorted'] = intxn_dc.apply(lambda x: tuple(sorted([x.STREET1SEGID, x.STREET2SEGID])), axis=1)
intxn_dc['geometry'] = intxn_dc.apply(lambda x: Point(x.LONGITUDE, x.LATITUDE), axis=1)

# compare intersection points with google maps
find out the meaning of intersection points is not likely as what I thought

In [23]:
segids = [4167, 8266]
segs_dc[segs_dc.STREETSEGID.isin(segids)][['STREETSEGID','DIRECTIONALITY','geometry']]

,STREETSEGID,DIRECTIONALITY,geometry
137,4167,0,LINESTRING (-77.01893411807097 38.905652137087...
5701,8266,1,LINESTRING (-77.01617612500391 38.905653722632...


In [25]:
segs_dc[segs_dc.STREETSEGID==8615]

,BEGINMEASURE,DIRECTIONALITY,ENDMEASURE,FACILITYID,FROMADDRESSLEFTTHEO,FROMADDRESSRIGHTTHEO,FROMNODEID,OBJECTID,OBJECTID_1,QUADRANT,...,SHAPE_Length,SOURCEID,STREETID,STREETSEGID,STREETTYPE,TOADDRESSLEFTTHEO,TOADDRESSRIGHTTHEO,TONODEID,UPDATETIMESTAMP,geometry
5702,0,2,84.072758,SEGID-8615,500,501,18272,5703,4074,NW,...,84.07273,15785050,105785,8615,ST,598,599,17178,2005-11-01T12:55:47,LINESTRING (-77.01893411807097 38.905652137087...


In [24]:
intxn_dc[(intxn_dc.STREET1SEGID.isin(segids))|(intxn_dc.STREET2SEGID.isin(segids))][['STREET1SEGID','STREET2SEGID']]

,STREET1SEGID,STREET2SEGID
4884,5326,4167
7283,8615,4167
11509,4167,8266
16658,8266,1457


In [10]:
unique_node_intxn = []
for nid, grp in intxn_dc.groupby('INTERSECTIONID'):
    geometry = grp.geometry.values[0]
    unique_node_intxn.append((nid,geometry.buffer(0.0001)))
unique_node_intxn = gp.GeoDataFrame(unique_node_intxn, columns=['INTERSECTIONID','geometry'])


In [13]:
intxn_dc[intxn_dc.INTERSECTIONID==-500]

,﻿X,Y,OBJECTID,MARID,INTERSECTIONID,STREET1ID,STREET2ID,ST1NAME,ST1TYPE,ST1QUAD,...,STREET1SEGID,STREET2SEGID,NODEID,INTERSECTION_TYPE,SOURCE,LATITUDE,LONGITUDE,latlon,stpair_sorted,geometry
8868,-77.007072,38.851531,8869,913131,-500,504845,508125,INTERSTATE 295,INTERSTATE,BN,...,4342,3579,-500,RAMP,DC GIS,38.851524,-77.00707,"(38.851524, -77.00707)","(3579, 4342)",POINT (-77.00707 38.851524)


In [12]:
from src.leaflet_creation import create_map_visualization

html_title = 'dc intersection'
file_path = 'data/'
file_name = 'dc int'
lon, lat  = -77.0908494, 38.9045525
zoom = 12
init_layers = ['streets']
map_layers = ['light','streets','satellite']
binding_data=[['stsg','street segment'],['intxn','intersection']]
segs_dc['color'] = '#a0a'
unique_node_intxn['color'] = '#0F0'
gpdfs = [segs_dc, unique_node_intxn]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)